In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df_train = pd.read_csv('../data/raw_train.csv')
df_test = pd.read_csv('../data/raw_test.csv')

df_dict = pd.read_excel('../data/Data Dictionary.xlsx')

In [3]:
def cleaner(dataframe, name):
    df = pd.DataFrame(dataframe.values, columns = [i.lower() for i in dataframe.columns])
    df['ltv'].apply(lambda i: i/100) # convert ltv to percentage
    
    # OHE
    pd.get_dummies(df, columns = ['branch_id', 'supplier_id', 'manufacturer_id', 'current_pincode_id'], drop_first = True)
    
    # nulls in employment indicate 'no history available'
    # fillna with -1, OHE for [Salaried : Self-employed : no employment history available]
    df.fillna(-1, inplace = True)
    df['employment_self'] = df['employment_type'].map(lambda i: 1 if i =='Self employed' else 0)
    df['employment_none'] = df['employment_type'].map(lambda i: 1 if i == -1 else 0)
    
    # convert string DD-MM-YYYY to datetime
    df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], dayfirst = True)
    df['disbursal_date'] = pd.to_datetime(df['disbursal_date'], dayfirst = True)
    df['age_at_disbursal'] = df['disbursal_date'].dt.year - df['date_of_birth'].dt.year
    
    # convert @yrs &mon string to int (@*12 + &) months
    df['average_acct_age_months'] = (df['average_acct_age'].str.extract("(\d+)yrs", expand=False).astype(int) * 12
                                     + df['average_acct_age'].str.extract("(\d+)mon", expand=False).astype(int))
    
    df['credit_history_length_months'] = (df['credit_history_length'].str.extract("(\d+)yrs", expand=False).astype(int) * 12
                                                + df['credit_history_length'].str.extract("(\d+)mon", expand=False).astype(int))
    
    # convert to ordinal scale
    df['cns_score_category'] = df['perform_cns_score_description'].map({'A-Very Low Risk': 1,
                                         'B-Very Low Risk': 2,
                                         'C-Very Low Risk': 3,
                                         'D-Very Low Risk': 4,
                                         'E-Low Risk': 5,
                                         'F-Low Risk': 6,
                                         'G-Low Risk':7 ,
                                         'H-Medium Risk': 8,
                                         'I-Medium Risk': 9,
                                         'J-High Risk': 10,
                                         'K-High Risk': 11,
                                         'L-Very High Risk': 12,
                                         'M-Very High Risk': 13,
                                         'No Bureau History Available': 0,
                                         'Not Scored: More than 50 active Accounts found': 0,
                                         'Not Scored: No Activity seen on the customer (Inactive)': 0,
                                         'Not Scored: No Updates available in last 36 months': 0,
                                         'Not Scored: Not Enough Info available on the customer': 0,
                                         'Not Scored: Only a Guarantor': 0,
                                         'Not Scored: Sufficient History Not Available': 0})
    # binary column for 'not scored due to unreliable history information'
    # in contrast with 'No bureau history information available' which is 0 on cns_score
    # 'not scored' are low but non-zero cns_score
    df['cns_score_unreliable'] = df['perform_cns_score_description'].map(lambda i: 1 if 'Not Scored:' in i else 0)
    
    # recast to int
    if 'loan_default' in df.columns:
        df['loan_default'] = df['loan_default'].astype(int)
    
    # drop 
    df.drop(columns = ['employment_type','date_of_birth','disbursal_date', 'average_acct_age', 'credit_history_length', 'perform_cns_score_description'], inplace = True)
    
    # save cleaned df to local
    df.to_csv(f'../data/clean_{name}.csv', index = False)

In [4]:
cleaner(df_train, 'train')
cleaner(df_test, 'test')

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
